In [1]:
##########################
# transfer learning using EfficientNetB7 as base model
# After training, the model shows more than 85% accuracy for validation dataset.
##########################

import numpy as np
from numpy import asarray
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import optimizers
#Use this to check if the GPU is configured correctly
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import * #Efficient Net included here
from tensorflow.keras import models
from tensorflow.keras import layers
import glob
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
import pathlib
from keras.preprocessing.image import ImageDataGenerator
import shutil
import pandas as pd
from tqdm import tqdm
print(device_lib.list_local_devices())

In [ ]:
# set parameters:  when batch_size is large, out of memory happens.
batch_size = 16
img_height = 224  #500
img_width = 224
NUMBER_OF_CLASSES=10

#count total number of images
data_dir = pathlib.Path("../input/vehicleImage/vehicles")
image_count = len(list(data_dir.glob('*/*.JPEG')))
print(image_count)

#The tree structure of the files can be used to compile a class_names list.
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)
class_id=range(len(class_names))
print(class_id)

In [ ]:
TRAIN_IMAGES_PATH = './train' 
VAL_IMAGES_PATH = './val' 
TEST_IMAGES_PATH = './test' 
External_DIR = '../input/vehicleImage/vehicles' 

os.makedirs(TRAIN_IMAGES_PATH, exist_ok = True)
os.makedirs(VAL_IMAGES_PATH, exist_ok = True)
os.makedirs(TEST_IMAGES_PATH, exist_ok = True)

# Create directories for each class.
for class_id in [x for x in range(len(class_names))]:
    os.makedirs(os.path.join(TRAIN_IMAGES_PATH, str(class_id)), exist_ok = True)
    os.makedirs(os.path.join(VAL_IMAGES_PATH, str(class_id)), exist_ok = True)
    os.makedirs(os.path.join(TEST_IMAGES_PATH, str(class_id)), exist_ok = True)  

In [ ]:
directory_contents = os.listdir('./val')
print(directory_contents)

for item in directory_contents:
    if os.path.isdir(item):
        print(item)

In [ ]:
# copy to train, validation and test directory
class_id=-1
for class_name in class_names:
    class_id=class_id+1
    Input_dir = '../input/vehicleImage/vehicles/'+class_name
    data_dir = pathlib.Path(Input_dir)
    image_count = len(list(data_dir.glob('*.JPEG')))
    #print(image_count)
    
    #get the list of names of all the images and shuffle
    list_ds = tf.data.Dataset.list_files( Input_dir + "/*", shuffle=False)
    list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)
    
    #Split the dataset into training, validation, and test sets :
    slice_size = int(image_count * 0.1)
    train_ds = list_ds.take(slice_size)
    val_test_ds = list_ds.skip(slice_size)
    val_ds = val_test_ds.take(slice_size)
    test_ds = val_test_ds.skip(slice_size)
    
    for item in  train_ds.as_numpy_iterator():
        shutil.copy(item.decode("utf-8"), os.path.join(TRAIN_IMAGES_PATH, str(class_id)))

    for item in  val_ds.as_numpy_iterator():
        shutil.copy(item.decode("utf-8"), os.path.join(VAL_IMAGES_PATH, str(class_id)))

    for item in  test_ds.as_numpy_iterator():
        shutil.copy(item.decode("utf-8"), os.path.join(TEST_IMAGES_PATH, str(class_id)))

    print(Input_dir)    

In [ ]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, 
# EfficientNetB3,... up to  7
# Higher the number, the more complex the model is. 
# and the larger resolutions it  can handle, but  the more GPU memory it will need
# loading pretrained conv base model
# input_shape is (height, width, number of channels) for images
input_shape=(img_height, img_width, 3)

conv_base = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    classes=NUMBER_OF_CLASSES,
    classifier_activation="softmax"
)


In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# avoid overfitting
# model.add(layers.Dropout(rate=0.2, name="dropout_out"))
model.add(layers.Dense(NUMBER_OF_CLASSES, activation="softmax", name="fc_out"))
conv_base.trainable = True

In [ ]:
# The ImageDataGenerator class allows us to specifiy 
# whatever augmentations we want so easily.
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
# Note that the validation data should not be augmented!
# and a very important step is to normalise the images through  rescaling
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    # This is the target directory
    TRAIN_IMAGES_PATH,
    # All images will be resized to target height and width.
    target_size=(img_height, img_width),
    batch_size=batch_size,
    # Since we use categorical_crossentropy loss, we need categorical labels
    class_mode="categorical"
)
validation_generator = test_datagen.flow_from_directory(
    VAL_IMAGES_PATH,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.RMSprop(learning_rate=2e-5),
    metrics=["acc"]
)


In [ ]:
#Train the model:
epochs=30
NUMBER_OF_TRAINING_IMAGES=1300
NUMBER_OF_VALIDATION_IMAGES=1300
history = model.fit(
    train_generator,
    steps_per_epoch=NUMBER_OF_TRAINING_IMAGES // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=NUMBER_OF_VALIDATION_IMAGES // batch_size,
    verbose=1,
    use_multiprocessing=True,
    workers=4
)

In [ ]:
# Visualize training results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# functions that convert a file path to an (img, label) pair:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
# get the list of names of all the images
test_ds1 = tf.data.Dataset.list_files(TEST_IMAGES_PATH+"/*/*", shuffle=False)
val_ds1 = tf.data.Dataset.list_files(VAL_IMAGES_PATH+"/*/*", shuffle=False)
AUTOTUNE = tf.data.AUTOTUNE
test_ds1 = test_ds1.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds1 = val_ds1.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
# predict validation data
plt.figure(figsize=(10, 10))
i=-1
for image, label in val_ds1.take(15):
    i=i+1
    print("Label: ", label.numpy())
    print(class_names[label])
    predictions = model.predict(np.expand_dims(image/255, axis=0))
    #score = tf.nn.softmax(predictions[0])
    score =predictions[0]
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    ax = plt.subplot(5, 3, i + 1)
    plt.tight_layout(pad=1.0)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(class_names[label] +": :"+ class_names[np.argmax(score)])

In [ ]:
# predict test data

plt.figure(figsize=(10, 10))
i=-1
for image, label in test_ds1.take(15):
    i=i+1
    print("Label: ", label.numpy())
    print(class_names[label])
    predictions = model.predict(np.expand_dims(image/255, axis=0))
    score = predictions[0]
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    plt.tight_layout(pad=1.0)
    ax = plt.subplot(5, 3, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(class_names[label] +": :"+ class_names[np.argmax(score)])

In [ ]:
##########################
# Vanilla model without enhanced techniques. about 50% accuracy for validation data
##########################

In [ ]:
import numpy as np
from numpy import asarray
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os
from sklearn.model_selection import StratifiedKFold
import pathlib
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
# set parameters
batch_size = 32
img_height = 224 #224  #500
img_width = 224

#count total number of images
data_dir = pathlib.Path("../input/vehicleImage/vehicles")
image_count = len(list(data_dir.glob('*/*.JPEG')))
print(image_count)

#get the list of names of all the images and shuffle
list_ds = tf.data.Dataset.list_files("../input/vehicleImage/vehicles/*/*", shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

#The tree structure of the files can be used to compile a class_names list.
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
#Split the dataset into training, validation, and test sets:
slice_size = int(image_count * 0.3)
train_ds = list_ds.take(slice_size)
val_test_ds = list_ds.skip(slice_size)
val_ds = val_test_ds.take(slice_size)
test_ds = val_test_ds.skip(slice_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())
print(tf.data.experimental.cardinality(test_ds).numpy())

In [ ]:
# functions that convert a file path to an (img, label) pair:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
# Use Dataset.map to create a dataset of image, label pairs:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
#Configure dataset for performance
#To be well shuffled.
#To be batched.
#Batches to be available as soon as possible.

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

#Visualize the data
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.title(class_names[label])
  plt.axis("off")

In [ ]:
num_classes = 10

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
#training the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
#Visualize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Visualize the data
image_batch, label_batch = next(iter(val_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
    predictions = model.predict(np.expand_dims(image_batch[i], axis=0))
    score = tf.nn.softmax(predictions[0])
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(class_names[label] +": :"+ class_names[np.argmax(score)])
    plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
i=-1
for image, label in test_ds.take(15):
    i=i+1
    print("Label: ", label.numpy())
    print(class_names[label])
    predictions = model.predict(np.expand_dims(image, axis=0))
    score = tf.nn.softmax(predictions[0])
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    ax = plt.subplot(5, 3, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(class_names[label] +": :"+ class_names[np.argmax(score)])

In [ ]:
##########################
# use pre-trained MobileNet without transfer learning.
# 95.7% accuracy for Aircraft_Carrier
# 93.2% accuracy for fireboat
# 100% accuracy for dozen of pictires I took.
##########################

import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras.layers.core import Dense, Activation
#from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.mobilenet import preprocess_input
#tf.keras.applications
import numpy as np
from IPython.display import Image

#from keras.applications import MobileNet


In [ ]:
mobile = tf.keras.applications.MobileNetV2()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
# predict Aircraft_Carrier: 1244/1300=95.7%
# get the list of names of all the images and shuffle
list_ds = tf.data.Dataset.list_files("../input/vehicleImage/vehicles/Aircraft_Carrier/*", shuffle=False)
count=0
correct=0
for f in list_ds:
    count=count+1
    preprocessed_image = prepare_image(f.numpy().decode("utf-8"))
    predictions = mobile.predict(preprocessed_image)
    results = imagenet_utils.decode_predictions(predictions)
    if results[0][0][1] == "aircraft_carrier" :
        correct=correct+1
print(count)
print(correct)

In [ ]:
# predict fireboat: 1212/1300=93.2%
# get the list of names of all the images and shuffle                                
list_ds = tf.data.Dataset.list_files("../input/vehicleImage/vehicles/Fireboat/*", shuffle=False)
count=0
correct=0                                    
                                     
count=0
correct=0
for f in list_ds:
    count=count+1
    preprocessed_image = prepare_image(f.numpy().decode("utf-8"))
    predictions = mobile.predict(preprocessed_image)
    results = imagenet_utils.decode_predictions(predictions)
    if results[0][0][1] == "fireboat" :
        correct=correct+1
print(count)
print(correct)

In [ ]:
preprocessed_image = prepare_image('../input/vehicleImage/vehicles/Fireboat/n03344393_10001.JPEG')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
print(results)
plt.imshow(preprocessed_image[0])
plt.title("predicted as:"+ results[0][0][1])

In [ ]:
preprocessed_image = prepare_image('../input/random/image/551eb20af2c474321f324598de76232.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
print(results)
plt.imshow(preprocessed_image[0])
plt.title("predicted as:"+ results[0][0][1])

In [ ]:
preprocessed_image = prepare_image('../input/random/image/f8e9ce997b5c2088926d2c157db25fa.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
print(results)
plt.imshow(preprocessed_image[0])
plt.title("predicted as:"+ results[0][0][1])